In [1]:
import numpy as np
import pandas as pd
import os
import pyodbc

from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    MetricType,
    Filter,
    FilterExpression,
    FilterExpressionList,
    QuotaStatus,
    GetMetadataRequest,
    RunReportRequest,
    OrderBy
)

import datetime
from datetime import datetime, timedelta, date

import warnings
warnings.simplefilter("ignore")

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\DARIEL\01 - Credenciales\CREDENCIALES GOOGLE\API GA4 MAYA YANBAL-8a9c90e8802e.json"

In [3]:
# FUNCIONES DE FORMATO

def sample_run_report(property_id):
    """Runs a metadata report on a Google Analytics 4 property."""

    client = BetaAnalyticsDataClient()

    request = GetMetadataRequest(name='properties/'+property_id+'/metadata')
    response = client.get_metadata(request)

    output =[]
    for dimension in response.dimensions:
        output.append({"Type": "Dimension", "API_Name": f"{dimension.api_name}", "UI_Name": f"{dimension.ui_name}", "Description": f"{dimension.description}", "Custom_definition": f"{dimension.custom_definition}", "Metric_type": "N/A"})

    for metric in response.metrics:
      output.append({"Type": "Metric", "API_Name": f"{metric.api_name}", "UI_Name": f"{metric.ui_name}", "Description": f"{metric.description}", "Custom_definition": f"{metric.custom_definition}", "Metric_type": f"{MetricType(metric.type_).name}"})
    
    df = pd.DataFrame(output)
    
    return df

def ga4_response_to_df(response):
    
    dim_len = len(response.dimension_headers)
    metric_len = len(response.metric_headers)
    all_data = []

    for row in response.rows:
        row_data = {}
        for i in range(0, dim_len):
            row_data.update({response.dimension_headers[i].name: row.dimension_values[i].value})
        for i in range(0, metric_len):
            row_data.update({response.metric_headers[i].name: row.metric_values[i].value})
        all_data.append(row_data)

    df = pd.DataFrame(all_data)
    
    return df

def get_ga4_report_df(property_id, dimensions, metrics, start_date, end_date):
    
    dimensions_ga4 = []
    for dimension in dimensions:
        dimensions_ga4.append(Dimension(name=dimension))

    metrics_ga4 = []
    for metric in metrics:
        metrics_ga4.append(Metric(name=metric))

    iter = 0
    offset = 0
    limit=250000

    while True:

        client = BetaAnalyticsDataClient()
        
        request = RunReportRequest(
            property = 'properties/'+property_id,
            return_property_quota = True,
            dimensions = dimensions_ga4,
            metrics = metrics_ga4,
            date_ranges = [DateRange(start_date=start_date, end_date=end_date)],
            dimension_filter=FilterExpression(
                and_group=FilterExpressionList(
                    expressions=[              
                        FilterExpression(
                            filter=Filter(
                                field_name="customUser:marker", 
                                string_filter=Filter.StringFilter(
                                    match_type=Filter.StringFilter.MatchType.PARTIAL_REGEXP, 
                                    value="(\d)|(\n)"
                                )
                            )
                        ),
                    ],          
                )
            ),
            limit=limit,
            offset=offset
        )
        
        response = client.run_report(request)
        
        if iter == 0:
            
            df_result = ga4_response_to_df(response)
            iter = iter + 1

        elif iter > 0:

            if not response.rows:
                break
            else:
                df_temp = ga4_response_to_df(response)
                df_result = pd.concat([df_result, df_temp], ignore_index=True)
                iter = iter + 1

        offset += limit
    
    return df_result

def cargarSQLSERVER(server, table_name, df, campaña):

    db = r'DWCorporacion'

    cadena_conexion = 'DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+db+';Trusted_Connection=yes'

    conn = pyodbc.connect(cadena_conexion)

    with conn.cursor() as cursor:

        for index, row in df.iterrows():
            row_values = row.values.tolist()
            placeholders = ', '.join(['?'] * len(row_values))
            query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(query, row_values)
            conn.commit()

    cursor = conn.cursor()
    sql_update = f"UPDATE {table_name} SET chrCodigoConsultora = NULL WHERE intNumeroCampania = '{campaña}';"
    cursor.execute(sql_update)
    conn.commit()
    
    conn.close()
    
def fn_sessions(property_id,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña):

    dimensions = ['customUser:marker']
    metrics = ['sessions']
    start_date = var_InicioCampania
    end_date = var_FinCampania

    df_GA4_Sesiones = get_ga4_report_df(property_id, dimensions, metrics, start_date, end_date)
    
    print(df_GA4_Sesiones.shape)
    
    df_GA4_Sesiones['Campania'] = var_NumeroCampaña
    df_GA4_Sesiones['chrCodigoConsultora'] = ''
    df_GA4_Sesiones['IntNumeroCampania'] = var_Campaña
    df_GA4_Sesiones['sessions'] = df_GA4_Sesiones['sessions'].astype('Int64')

    print(df_GA4_Sesiones.shape)
    
    return df_GA4_Sesiones

def fn_herramientas(property_id,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña):

    dimensions = ['customUser:marker', 'eventName', 'customEvent:event_label']
    metrics = ['eventCount']
    start_date = var_InicioCampania
    end_date = var_FinCampania

    df_GA4_DentroHerramienta = get_ga4_report_df(property_id, dimensions, metrics, start_date, end_date)

    print(df_GA4_DentroHerramienta.shape)

    #   Primer Filtro  
    df_GA4_DentroHerramienta = df_GA4_DentroHerramienta[df_GA4_DentroHerramienta['eventName']=='view_DL_item']
    df_GA4_DentroHerramienta = df_GA4_DentroHerramienta[df_GA4_DentroHerramienta['customEvent:event_label']!='']
    df_GA4_DentroHerramienta.reset_index(drop=True, inplace=True)

    # Agrupar
    df_GA4_DentroHerramienta['eventCount'] = df_GA4_DentroHerramienta['eventCount'].astype(int)
    df_GA4_DentroHerramienta = df_GA4_DentroHerramienta.groupby('customUser:marker', as_index =False)['eventCount'].sum()
    df_GA4_DentroHerramienta = df_GA4_DentroHerramienta.sort_values(by=['eventCount'], ascending=False)

    df_GA4_DentroHerramienta.reset_index(drop=True, inplace=True)
    df_GA4_DentroHerramienta['Campania'] = var_NumeroCampaña
    df_GA4_DentroHerramienta['chrCodigoConsultora'] = ''
    df_GA4_DentroHerramienta['IntNumeroCampania'] = var_Campaña
    df_GA4_DentroHerramienta['eventCount'] = df_GA4_DentroHerramienta['eventCount'].astype('Int64')
    
    print(df_GA4_DentroHerramienta.shape)

    return df_GA4_DentroHerramienta

def fn_descargas(property_id,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña):
    
    dimensions = ['customUser:marker', 'eventName', 'customEvent:event_label', 'customEvent:folder_name']
    metrics = ['eventCount']
    start_date = var_InicioCampania
    end_date = var_FinCampania

    df_GA4_Descargas = get_ga4_report_df(property_id, dimensions, metrics, start_date, end_date)
    
    print(df_GA4_Descargas.shape)

    #   Primer Filtro
    df_GA4_Descargas = df_GA4_Descargas[df_GA4_Descargas['eventName']=='download_DL_item']
    df_GA4_Descargas.drop(['eventName'], axis=1, inplace=True)

    #   Segundo Filtro
    items_list = ['null (0)', '(notset)', '']
    df_GA4_Descargas = df_GA4_Descargas[~df_GA4_Descargas['customEvent:event_label'].isin(items_list)]
    df_GA4_Descargas.reset_index(drop=True, inplace=True)

    df_GA4_Descargas['Campania'] = var_NumeroCampaña
    df_GA4_Descargas['chrCodigoConsultora'] = ''
    df_GA4_Descargas['IntNumeroCampania'] = var_Campaña
    df_GA4_Descargas['eventCount'] = df_GA4_Descargas['eventCount'].astype('Int64')
    
    df_GA4_Descargas  = df_GA4_Descargas[['customUser:marker','eventCount','Campania','chrCodigoConsultora','customEvent:event_label','customEvent:folder_name','IntNumeroCampania']]

    print(df_GA4_Descargas.shape)

    return df_GA4_Descargas

def fn_filtrarCarpetas(df, list_carpetas):
    
    str_list = '|'.join(list_carpetas)
    df['Conditional'] = df['customEvent:folder_name'].str.contains(str_list)

    df = df[df['Conditional'] == True]

    df.drop(['Conditional'], axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

def fn_reporte(property_id,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña):
    
    dimensions = ['customUser:marker', 'unifiedPagePathScreen']
    metrics = ['eventCount']
    start_date = var_InicioCampania
    end_date = var_FinCampania

    df_GA4_VieronReporte = get_ga4_report_df(property_id, dimensions, metrics, start_date, end_date)

    print(df_GA4_VieronReporte.shape)

    #   Primer Filtro 
    df_GA4_VieronReporte['Conditional'] = df_GA4_VieronReporte['unifiedPagePathScreen'].str.contains('eport')
    df_GA4_VieronReporte = df_GA4_VieronReporte[df_GA4_VieronReporte['Conditional'] == True]

    df_GA4_VieronReporte.drop(['Conditional'], axis=1, inplace=True)
    df_GA4_VieronReporte.reset_index(drop=True, inplace=True)

    df_GA4_VieronReporte['Campania'] = var_NumeroCampaña
    df_GA4_VieronReporte['chrCodigoConsultora'] = ''
    df_GA4_VieronReporte['IntNumeroCampania'] = var_Campaña
    df_GA4_VieronReporte['eventCount'] = df_GA4_VieronReporte['eventCount'].astype('Int64')
    
    df_GA4_VieronReporte  = df_GA4_VieronReporte[['customUser:marker','eventCount','Campania','chrCodigoConsultora','IntNumeroCampania','unifiedPagePathScreen']]

    print(df_GA4_VieronReporte.shape)

    return df_GA4_VieronReporte


In [ ]:
'''
df_ReporteDiccionario = sample_run_report(property_id)

#   Se crea un archivo Excell con la lista de todas las métricas a exportar
df_ReporteDiccionario.to_excel(f'Lista de Métricas y Dimensiones a Exportar_{str(date.today())}.xlsx', sheet_name = 'GA4_report', engine = 'xlsxwriter', index=False, header=True)

df_ReporteDiccionario.head()
'''

##  BASE DE DATOS

In [4]:
per_ds_srv = r'PERMS03\PERU'
per_ds_db = r'DWUniqueYanbal'

per_cadena_coneccion = 'DRIVER={SQL Server Native Client 11.0};SERVER='+per_ds_srv+';DATABASE='+per_ds_db+';Trusted_Connection=yes'

#   CONSULTAR FECHA INICIO Y FIN DE CAMPANIA ACTUAL

per_conn = pyodbc.connect(per_cadena_coneccion)

query_infoCampania = '''
SELECT 
    intNumeroCampania, 
    'S'+CAST(smlNumeroSemanaCampania AS VARCHAR) smlNumeroSemanaCampania,
    smlNumeroCampania, 
    dtmFechaInicioSemana, 
    dtmFechaFinSemana
FROM DWUniqueYanbal.dbo.Tiempo
WHERE intNumeroCampania = (
    SELECT 
        intNumeroCampania
    FROM DWUniqueYanbal.dbo.Tiempo
    WHERE dtmFechaInicioSemana <= GETDATE() AND dtmFechaFinSemana >= GETDATE()
)
'''

df_infoCampania = pd.read_sql_query(query_infoCampania, per_conn)

per_conn.close()


df_infoCampania['smlNumeroCampania'] = df_infoCampania['smlNumeroCampania'].astype(str)
df_infoCampania['dtmFechaInicioSemana'] = df_infoCampania['dtmFechaInicioSemana'].dt.strftime('%Y-%m-%d')
df_infoCampania['dtmFechaFinSemana'] = df_infoCampania['dtmFechaFinSemana'].dt.strftime('%Y-%m-%d')

var_InicioCampania = df_infoCampania.loc[0, 'dtmFechaInicioSemana']
var_FinCampania = df_infoCampania.loc[3, 'dtmFechaFinSemana']
var_Campaña = df_infoCampania.loc[0, 'intNumeroCampania']
var_NumeroCampaña = df_infoCampania.loc[0, 'smlNumeroCampania']

print("Campaña: " + str(var_Campaña))
print("Numero Campaña: " + var_NumeroCampaña)
print("Rango: " + var_InicioCampania + " - " + var_FinCampania)

Campaña: 202308
Numero Campaña: 8
Rango: 2023-07-15 - 2023-08-11


##  SESIONES

In [5]:
#   id reporte GA
id_Peru = '248018826'
id_Colombia = '247990848'
id_Bolivia = '248009338'
id_Ecuador = '255044946'
id_Guatemala = '257732827'
id_Mexico = '255000471'

#   NO SE USAN
id_USA = '247996799'
id_España = '248042891'
id_Italia = '274127600'

In [ ]:
df_session_peru         = fn_sessions(id_Peru,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_session_colombia     = fn_sessions(id_Colombia,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_session_bolivia      = fn_sessions(id_Bolivia,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_session_ecuador      = fn_sessions(id_Ecuador,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_session_guatemala    = fn_sessions(id_Guatemala,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_session_mexico       = fn_sessions(id_Mexico,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)

In [ ]:
'''
df_session_peru.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_session_peru.to_csv(r"C:\DARIEL\02 - Maya\C8\PERU\01 - marker_for_session_PERU_C8_2023")

df_session_colombia.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_session_colombia.to_csv(r"C:\DARIEL\02 - Maya\C8\COLOMBIA\01 - marker_for_session_COLOMBIA_C8_2023")

df_session_bolivia.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_session_bolivia.to_csv(r"C:\DARIEL\02 - Maya\C8\BOLIVIA\01 - marker_for_session_BOLIVIA_C8_2023")

df_session_ecuador.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_session_ecuador.to_csv(r"C:\DARIEL\02 - Maya\C8\ECUADOR\01 - marker_for_session_ECUADOR_C8_2023")

df_session_guatemala.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_session_guatemala.to_csv(r"C:\DARIEL\02 - Maya\C8\GUATEMALA\01 - marker_for_session_GUATEMALA_C8_2023")

df_session_mexico.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_session_mexico.to_csv(r"C:\DARIEL\02 - Maya\C8\MEXICO\01 - marker_for_session_MEXICO_C8_2023")
'''

In [ ]:
'''
cargarSQLSERVER(r'PERMS03\PERU','[DWCorporacion].dbo.GA_Sesion_hst',df_session_peru,var_Campaña)
cargarSQLSERVER(r'SRVDATAX\COLOMBIA','[DWCorporacion].dbo.GA_Sesion_hst',df_session_colombia,var_Campaña)
cargarSQLSERVER(r'PERMS11\BOLIVIA','[DWCorporacion].dbo.GA_Sesion_hst',df_session_bolivia,var_Campaña)
cargarSQLSERVER(r'ECUSRVDW0\ECUA','[DWCorporacion].dbo.GA_Sesion_hst',df_session_ecuador,var_Campaña)
'''

##  DENTRO DE HERRAMIENTA

In [ ]:
df_herramienta_peru         = fn_herramientas(id_Peru,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_herramienta_colombia     = fn_herramientas(id_Colombia,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_herramienta_bolivia      = fn_herramientas(id_Bolivia,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_herramienta_ecuador      = fn_herramientas(id_Ecuador,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_herramienta_guatemala    = fn_herramientas(id_Guatemala,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_herramienta_mexico       = fn_herramientas(id_Mexico,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)

In [ ]:
'''
df_herramienta_peru.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_herramienta_peru.to_csv(r"C:\DARIEL\02 - Maya\C8\PERU\02 - marker_for_herramientas_PERU_C8_2023")

df_herramienta_colombia.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_herramienta_colombia.to_csv(r"C:\DARIEL\02 - Maya\C8\COLOMBIA\02 - marker_for_herramientas_COLOMBIA_C8_2023")

df_herramienta_bolivia.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_herramienta_bolivia.to_csv(r"C:\DARIEL\02 - Maya\C8\BOLIVIA\02 - marker_for_herramientas_BOLIVIA_C8_2023")

df_herramienta_ecuador.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_herramienta_ecuador.to_csv(r"C:\DARIEL\02 - Maya\C8\ECUADOR\02 - marker_for_herramientas_ECUADOR_C8_2023")

df_herramienta_guatemala.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_herramienta_guatemala.to_csv(r"C:\DARIEL\02 - Maya\C8\GUATEMALA\02 - marker_for_herramientas_GUATEMALA_C8_2023")

df_herramienta_mexico.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_herramienta_mexico.to_csv(r"C:\DARIEL\02 - Maya\C8\MEXICO\02 - marker_for_herramientas_MEXICO_C8_2023")
'''

In [ ]:
'''
cargarSQLSERVER(r'PERMS03\PERU','[DWCorporacion].dbo.GA_Herramientas_hst',df_Herramienta_peru,var_Campaña)
cargarSQLSERVER(r'SRVDATAX\COLOMBIA','[DWCorporacion].dbo.GA_Herramientas_hst',df_Herramienta_colombia,var_Campaña)
cargarSQLSERVER(r'PERMS11\BOLIVIA','[DWCorporacion].dbo.GA_Herramientas_hst',df_Herramienta_bolivia,var_Campaña)
cargarSQLSERVER(r'ECUSRVDW0\ECUA','[DWCorporacion].dbo.GA_Herramientas_hst',df_Herramienta_ecuador,var_Campaña)
'''

##  DESCARGAS

In [ ]:
df_descargas_peru       = fn_descargas(id_Peru,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_descargas_colombia   = fn_descargas(id_Colombia,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_descargas_bolivia    = fn_descargas(id_Bolivia,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_descargas_ecuador    = fn_descargas(id_Ecuador,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_descargas_guatemala  = fn_descargas(id_Guatemala,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_descargas_mexico     = fn_descargas(id_Mexico,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)

In [ ]:
carpetas_TablaLisbet = ['compartir', 'Semana']

df_GA4_DescargaronRRSS_peru = fn_filtrarCarpetas(df_descargas_peru, carpetas_TablaLisbet)
df_GA4_DescargaronRRSS_colombia = fn_filtrarCarpetas(df_descargas_colombia, carpetas_TablaLisbet)
df_GA4_DescargaronRRSS_bolivia = fn_filtrarCarpetas(df_descargas_bolivia, carpetas_TablaLisbet)
df_GA4_DescargaronRRSS_ecuador = fn_filtrarCarpetas(df_descargas_ecuador, carpetas_TablaLisbet)
df_GA4_DescargaronRRSS_guatemala = fn_filtrarCarpetas(df_descargas_guatemala, carpetas_TablaLisbet)
df_GA4_DescargaronRRSS_mexico = fn_filtrarCarpetas(df_descargas_mexico, carpetas_TablaLisbet)

In [ ]:
'''
df_GA4_DescargaronRRSS_peru.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_GA4_DescargaronRRSS_peru['eventName'] = 'download_DL_item'
df_GA4_DescargaronRRSS_peru = df_GA4_DescargaronRRSS_peru[['customUser:marker', 'eventName','customEvent:event_label','customEvent:folder_name','eventCount']]
df_GA4_DescargaronRRSS_peru.to_csv(r"C:\DARIEL\02 - Maya\C8\PERU\03 - marker_for_Redes_sociales_PERU_C8_2023")

df_GA4_DescargaronRRSS_colombia.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_GA4_DescargaronRRSS_colombia['eventName'] = 'download_DL_item'
df_GA4_DescargaronRRSS_colombia = df_GA4_DescargaronRRSS_colombia[['customUser:marker', 'eventName','customEvent:event_label','customEvent:folder_name','eventCount']]
df_GA4_DescargaronRRSS_colombia.to_csv(r"C:\DARIEL\02 - Maya\C8\COLOMBIA\03 - marker_for_Redes_sociales_COLOMBIA_C8_2023")

df_GA4_DescargaronRRSS_bolivia.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_GA4_DescargaronRRSS_bolivia['eventName'] = 'download_DL_item'
df_GA4_DescargaronRRSS_bolivia = df_GA4_DescargaronRRSS_bolivia[['customUser:marker', 'eventName','customEvent:event_label','customEvent:folder_name','eventCount']]
df_GA4_DescargaronRRSS_bolivia.to_csv(r"C:\DARIEL\02 - Maya\C8\BOLIVIA\03 - marker_for_Redes_sociales_BOLIVIA_C8_2023")

df_GA4_DescargaronRRSS_ecuador.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_GA4_DescargaronRRSS_ecuador['eventName'] = 'download_DL_item'
df_GA4_DescargaronRRSS_ecuador = df_GA4_DescargaronRRSS_ecuador[['customUser:marker', 'eventName','customEvent:event_label','customEvent:folder_name','eventCount']]
df_GA4_DescargaronRRSS_ecuador.to_csv(r"C:\DARIEL\02 - Maya\C8\ECUADOR\03 - marker_for_Redes_sociales_ECUADOR_C8_2023")

df_GA4_DescargaronRRSS_guatemala.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_GA4_DescargaronRRSS_guatemala['eventName'] = 'download_DL_item'
df_GA4_DescargaronRRSS_guatemala = df_GA4_DescargaronRRSS_guatemala[['customUser:marker', 'eventName','customEvent:event_label','customEvent:folder_name','eventCount']]
df_GA4_DescargaronRRSS_guatemala.to_csv(r"C:\DARIEL\02 - Maya\C8\GUATEMALA\03 - marker_for_Redes_sociales_GUATEMALA_C8_2023")

df_GA4_DescargaronRRSS_mexico.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_GA4_DescargaronRRSS_mexico['eventName'] = 'download_DL_item'
df_GA4_DescargaronRRSS_mexico = df_GA4_DescargaronRRSS_mexico[['customUser:marker', 'eventName','customEvent:event_label','customEvent:folder_name','eventCount']]
df_GA4_DescargaronRRSS_mexico.to_csv(r"C:\DARIEL\02 - Maya\C8\MEXICO\03 - marker_for_Redes_sociales_MEXICO_C8_2023")
'''

In [ ]:
'''
cargarSQLSERVER(r'PERMS03\PERU','[DWCorporacion].dbo.GA_RRSS_hst', df_GA4_DescargaronRRSS_peru, var_Campaña)
cargarSQLSERVER(r'SRVDATAX\COLOMBIA','[DWCorporacion].dbo.GA_RRSS_hst', df_GA4_DescargaronRRSS_colombia, var_Campaña)
cargarSQLSERVER(r'PERMS11\BOLIVIA','[DWCorporacion].dbo.GA_RRSS_hst', df_GA4_DescargaronRRSS_bolivia, var_Campaña)
cargarSQLSERVER(r'ECUSRVDW0\ECUA','[DWCorporacion].dbo.GA_RRSS_hst', df_GA4_DescargaronRRSS_ecuador, var_Campaña)
'''

##  QUE VIERON REPORTE

In [ ]:
df_reporte_peru         = fn_reporte(id_Peru,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_reporte_colombia     = fn_reporte(id_Colombia,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_reporte_bolivia      = fn_reporte(id_Bolivia,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_reporte_ecuador      = fn_reporte(id_Ecuador,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_reporte_guatemala    = fn_reporte(id_Guatemala,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)
df_reporte_mexico       = fn_reporte(id_Mexico,var_InicioCampania,var_FinCampania,var_NumeroCampaña,var_Campaña)

In [ ]:
'''
df_reporte_peru.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_reporte_peru = df_reporte_peru[['unifiedPagePathScreen', 'customUser:marker', 'eventCount']]
df_reporte_peru.to_csv(r"C:\DARIEL\02 - Maya\C8\PERU\04 - Reporte que vieron reportes_PERU_C8_2023")

df_reporte_colombia.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_reporte_colombia = df_reporte_colombia[['unifiedPagePathScreen', 'customUser:marker', 'eventCount']]
df_reporte_colombia.to_csv(r"C:\DARIEL\02 - Maya\C8\COLOMBIA\04 - Reporte que vieron reportes_COLOMBIA_C8_2023")

df_reporte_bolivia.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_reporte_bolivia = df_reporte_bolivia[['unifiedPagePathScreen', 'customUser:marker', 'eventCount']]
df_reporte_bolivia.to_csv(r"C:\DARIEL\02 - Maya\C8\BOLIVIA\04 - Reporte que vieron reportes_BOLIVIA_C8_2023")

df_reporte_ecuador.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_reporte_ecuador = df_reporte_ecuador[['unifiedPagePathScreen', 'customUser:marker', 'eventCount']]
df_reporte_ecuador.to_csv(r"C:\DARIEL\02 - Maya\C8\ECUADOR\04 - Reporte que vieron reportes_ECUADOR_C8_2023")

df_reporte_guatemala.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_reporte_guatemala = df_reporte_guatemala[['unifiedPagePathScreen', 'customUser:marker', 'eventCount']]
df_reporte_guatemala.to_csv(r"C:\DARIEL\02 - Maya\C8\GUATEMALA\04 - Reporte que vieron reportes_GUATEMALA_C8_2023")

df_reporte_mexico.drop(['Campania','chrCodigoConsultora','IntNumeroCampania'], axis=1, inplace=True)
df_reporte_mexico = df_reporte_mexico[['unifiedPagePathScreen', 'customUser:marker', 'eventCount']]
df_reporte_mexico.to_csv(r"C:\DARIEL\02 - Maya\C8\MEXICO\04 - Reporte que vieron reportes_MEXICO_C8_2023")
'''

In [ ]:
'''
cargarSQLSERVER(r'PERMS03\PERU','GA_Reporte_hst',df_reporte_peru,var_Campaña)
cargarSQLSERVER(r'SRVDATAX\COLOMBIA','GA_Reporte_hst',df_reporte_colombia,var_Campaña)
cargarSQLSERVER(r'PERMS11\BOLIVIA','GA_Reporte_hst',df_reporte_bolivia,var_Campaña)
cargarSQLSERVER(r'ECUSRVDW0\ECUA','GA_Reporte_hst',df_reporte_ecuador,var_Campaña)
'''